In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import json
import os

class VirtualSportsCollector:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
            'Accept': 'application/json, text/plain, */*',
            'Accept-Encoding': 'gzip, deflate, br, zstd',
            'Accept-Language': 'en-US,en;q=0.6',
            'Origin': 'https://www.eurobet.it',
            'X-EB-Accept-Language': 'it_IT',
            'X-EB-MarketId': '5',
            'X-EB-PlatformId': '1',
            'Connection': 'keep-alive'
        }
        self.base_url = "https://virtualservice.eurobet.it/virtual-winning-service/virtual-schedule/services/winningresult/55/22/{}"
        self.csv_filename = "virtual_matches_data.csv"
        self.excel_filename = "virtual_matches_data.xlsx"

    def create_match_id(self, row):
        """Create a unique identifier for each match"""
        return f"{row['date']}_{row['hour']}_{row['home_team']}_{row['away_team']}"

    def load_existing_data(self):
        """Load existing data from CSV if it exists"""
        if os.path.exists(self.csv_filename):
            return pd.read_csv(self.csv_filename)
        return pd.DataFrame()

    def get_virtual_data(self, start_date, end_date):
        all_matches = []
        current_date = start_date

        while current_date <= end_date:
            date_str = current_date.strftime("%d-%m-%Y")
            url = self.base_url.format(date_str)

            try:
                response = requests.get(url, headers=self.headers)
                if response.status_code == 200:
                    data = response.json()
                    if 'result' in data and 'groupDate' in data['result']:
                        for group in data['result']['groupDate']:
                            for event in group['events']:
                                match_data = {
                                    'date': event['date'],
                                    'hour': event['hour'],
                                    'home_team': event['eventDescription'].split(' - ')[0],
                                    'away_team': event['eventDescription'].split(' - ')[1],
                                    'score': event['finalResult'],
                                    'home_goals': int(event['finalResult'].split('-')[0]),
                                    'away_goals': int(event['finalResult'].split('-')[1]),
                                    'datetime': pd.to_datetime(f"{event['date']} {event['hour']}", format='%d-%m-%Y %H:%M:%S')
                                }

                                for odd_group in event['oddGroup']:
                                    if odd_group['betDescriptionAbbr'] == '1X2':
                                        match_data['odds_1'] = odd_group['odds'][0]
                                        match_data['result'] = odd_group['resultDescription'][0]
                                    elif odd_group['betDescriptionAbbr'] == 'U/O 2.5':
                                        match_data['over_under_25'] = odd_group['resultDescription'][0]
                                        match_data['odds_over_under_25'] = odd_group['odds'][0]
                                    elif odd_group['betDescriptionAbbr'] == 'Goal/No Goal':
                                        match_data['goal_no_goal'] = odd_group['resultDescription'][0]
                                        match_data['odds_goal_no_goal'] = odd_group['odds'][0]

                                all_matches.append(match_data)

                time.sleep(1)  # Respect rate limiting
            except Exception as e:
                print(f"Error fetching data for {date_str}: {e}")

            current_date += timedelta(days=1)

        return pd.DataFrame(all_matches)

    def merge_and_save_data(self, new_data):
        """Merge new data with existing data, remove duplicates, and save"""
        existing_data = self.load_existing_data()

        if not existing_data.empty:
            # Convert datetime column in existing data if it's not already datetime
            existing_data['datetime'] = pd.to_datetime(existing_data['datetime'])

        # Combine existing and new data
        combined_data = pd.concat([existing_data, new_data], ignore_index=True)

        # Create unique identifier for each match
        combined_data['match_id'] = combined_data.apply(self.create_match_id, axis=1)

        # Remove duplicates based on match_id
        combined_data = combined_data.drop_duplicates(subset=['match_id'], keep='first')

        # Sort by datetime in descending order (most recent first)
        combined_data = combined_data.sort_values('datetime', ascending=False)

        # Drop the match_id column as it's no longer needed
        combined_data = combined_data.drop('match_id', axis=1)

        # Save to CSV and Excel
        combined_data.to_csv(self.csv_filename, index=False)
        combined_data.to_excel(self.excel_filename, index=False)

        return combined_data

    def collect_data(self, days_back=90):
        """Main method to collect and process data"""
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days_back)

        print(f"Collecting data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

        new_data = self.get_virtual_data(start_date, end_date)
        if not new_data.empty:
            final_data = self.merge_and_save_data(new_data)
            print(f"Data saved successfully")
            print(f"Total matches in database: {len(final_data)}")
            print(f"Files saved as: {self.csv_filename} and {self.excel_filename}")
        else:
            print("No new data collected")

def main(days_back=90):
    collector = VirtualSportsCollector()
    collector.collect_data(days_back)

if __name__ == "__main__":
    main()  # Default 90 days